<a href="https://colab.research.google.com/github/jsebastianquiroga/DEEP_LEARNING_UPJ/blob/main/taller_dos/ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset
  - class1
    - image1.jpg
    - image2.jpg
  - class2
    - image1.jpg
    - image2.jpg
    

In [ ]:
import os
import io
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
class ImageLoader:
    def __init__(self, folder_id):
        self.folder_id = folder_id
        self.service = self.get_drive_service()
        self.images, self.labels = self.load_data_from_drive(target_size=(224, 224))

    def get_drive_service(self):
        creds = None
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json')
        service = build('drive', 'v3', credentials=creds)
        return service

    def get_folder_id(self, folder_id):
        try:
            folder = self.service.files().get(fileId=folder_id).execute()
            if folder['mimeType'] == 'application/vnd.google-apps.folder':
                return folder_id
            else:
                print(f'The given ID is not a folder.')
                return None
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def get_folders_images(self, parent_id):
        query = f"'{parent_id}' in parents and mimeType != 'application/vnd.google-apps.folder' and trashed = false"
        results = self.service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])
        return items

    def read_image_from_drive(self, file_id):
        request = self.service.files().get_media(fileId=file_id)
        image_data = io.BytesIO()
        downloader = MediaIoBaseDownload(image_data, request)
        done = False
        while done is False:
            _, done = downloader.next_chunk()
        image_data.seek(0)
        image_array = np.asarray(bytearray(image_data.read()), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        return image

    def load_data_from_drive(self, target_size=(224, 224)):
        parent_id = self.get_folder_id(self.folder_id)
        if parent_id is None:
            return None

        class_folders = self.get_folders_images(parent_id)
        images, labels = [], []
        for folder in class_folders:
            folder_id = folder['id']
            folder_name = folder['name']
            images_ids = self.get_folders_images(folder_id)
            for image_meta in images_ids:
                image = self.read_image_from_drive(image_meta['id'])
                if image is not None:
                    resized_image = cv2.resize(image, target_size)
                    image_array = img_to_array(resized_image)
                    images.append(image_array)
                    labels.append(folder_name)
        return np.array(images), np.array(labels)

In [ ]:
if __name__ == '__main__':
    folder_url = 'https://drive.google.com/drive/folders/1l6T6jPw6EE-fcCy7ChgeMdYA7bNawHk4'
    folder_id = folder_url.split('/')[-1]
    image_loader = ImageLoader(folder_id)

    images = image_loader.images
    labels = image_loader.labels

    # Split your data into training and validation sets, preprocess it, and train your model...
